In [ ]:
#| default_exp parser

# parser

> Parse a SQL query.

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from collections import defaultdict
from sqlglot import parse_one
from sqlglot.expressions import Expression
from sqlviz.utils import *

In [ ]:
#| export
def parse_operations(sql:str) -> dict:
    
    "Given a query string, return a dictionary of parsed SQL operations."
    
    ops = defaultdict(lambda: list())
    
    ast = parse_one(sql)
    for exp in ast.walk():
        item, parent, key = exp
        if parent == ast:
            ops[key].append(item.sql())
    
    ops['select'] = ops['expressions']
    ops.pop('expressions')
    
    try: ops['limit'] = ops['limit'][0][6:]
    except: pass
    try: ops['from'] = ops['from'][0][5:]
    except: pass
    try: ops['joins'] = ops['joins']
    except: pass
    try: ops['where'] = ops['where'][0][6:]
    except: pass
    try: ops['group'] = ops['group'][0][9:]
    except: pass
    try: ops['having'] = ops['having'][0][7:]
    except: pass
    try: ops['order'] = ops['order'][0][9:]
    except: pass
    
    res = {}
    for k,v in ops.items():
        if v == []: pass
        else: res[k] = v
            
    res['_ops'] = list(res.keys())
    
    return res

In [ ]:
from sqlviz.sql import *

for q in Q:
    for k,v in parse_operations(q).items():
        print(f"{k}: {v}")
    print("")

select: ['1']
_ops: ['select']

limit: 5
from: pageviews
select: ['email', 'url']
_ops: ['limit', 'from', 'select']

limit: 5
from: pageviews AS pv
joins: ['JOIN users AS u USING (user_id)', 'LEFT JOIN accounts AS a ON (u.account_id = a.id)']
where: c5 > 0 AND NOT c6 IS NULL
group: c1
having: COUNT(DISTINCT things) > 1
order: COUNT(DISTINCT things)
select: ['c1', 'c2 AS "foo"', 'c3 + c4 AS "bar"', 'COUNT(DISTINCT things)']
_ops: ['limit', 'from', 'joins', 'where', 'group', 'having', 'order', 'select']

from: opportunity_raw
where: iswon AND amount > 0
select: ['accountid AS account_id', 'closedate AS close_date', 'company_industry_group__c AS industry', 'amount']
_ops: ['from', 'where', 'select']

